In [ ]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer
import multiprocessing as mp
from models.TorchAvitm import TorchAvitm
from gensim.models import Word2Vec

In [2]:
# Load dataset
dataset = Dataset()
dataset.load("preprocessed_datasets/newsgroup/newsgroup_lemmatized_5")

False

In [3]:
# Load model
model = TorchAvitm()

In [4]:
# Set model hyperparameters
model.hyperparameters['n_components'] = 5

In [12]:
model.hyperparameters

{'n_components': 5}

In [5]:
# Topic diversity
topic_diversity = Topic_diversity()

# KL_Uniform
#kl_uniform = KL_uniform()

In [6]:
# Define optimization parameters
opt_params = {}
opt_params["n_calls"] = 30
opt_params["minimizer"] = forest_minimize
opt_params["different_iteration"] = 3
opt_params["n_random_starts"] = 5
#opt_params["extra_metrics"] = [kl_uniform] # List of extra metrics
opt_params["n_jobs"] = mp.cpu_count() -1 # Enable multiprocessing
opt_params["verbose"] = True
opt_params["save_path"] = "results" #create folder if it doesn't exist 

In [7]:
# Create search space for optimization
search_space = {
    "num_epochs": Real(low=1, high=50),
    #"eta": Real(low=0.01, high=5.0)
}

In [8]:
# Initialize optimizer
optimizer = Optimizer(
    model,
    dataset,
    topic_diversity,
    search_space,
    opt_params)

In [9]:
# Disable computing of topic document matrix to optimize performance
optimizer.topic_document_matrix = False
optimizer.topic_word_matrix = False

In [10]:
# Optimize
res = optimizer.optimize()

print(res.hyperparameters) # Best values for the hyperparameters
print(res.function_values) # Score of the optimized metric
print("Optimized metric: "+res.optimized_metric)

------------------------------------------
------------------------------------------
Bayesian optimization parameters:
-n_calls:  30 
-different_iteration:  3 
-n_random_starts:  5 
-minimizer:  forest_minimize 
-acq_func:  LCB 
-kernel:  1**2 * Matern(length_scale=1, nu=1.5)
------------------------------------------
Iteration No: 1 started. Evaluating function at random point.


KeyError: 'n_components'